In [ ]:
import boto3
import time
import pandas as pd
import numpy as np
import pymysql as mysql
import os
from datetime import datetime
import warnings
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import joblib

# Suppress all warnings
warnings.filterwarnings("ignore")

# AWS Credentials & Region
AWS_REGION = "us-east-2"  # Change to your region
DATABASE = "chalk"
TABLE = "chalkjuice_data"
S3_OUTPUT = "s3://chalkjuice/golden_athena/"  # Replace with your actual S3 bucket

# Initialize Athena Client
athena_client = boto3.client("athena", region_name=AWS_REGION)

weights = [5,7,9,13,.3,.25,.25,.2]
games_back = weights[0] + weights[1] + weights[2] + weights[3] + 1


# AWS Credentials & Region
AWS_REGION = "us-east-2"  # Change to your region
DATABASE = "chalk"
TABLE = "chalkjuice_data"
S3_OUTPUT = "s3://chalkjuice/golden_athena/"  # Replace with your actual S3 bucket

# Initialize Athena Client
athena_client = boto3.client("athena", region_name=AWS_REGION)

In [4]:
def query_athena_df(query):
    # Start Query Execution
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={"Database": DATABASE},
        ResultConfiguration={"OutputLocation": S3_OUTPUT},
    )

    # Get Query Execution ID
    query_execution_id = response["QueryExecutionId"]

    # Wait for Query to Complete
    while True:
        status = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        state = status["QueryExecution"]["Status"]["State"]
        
        if state in ["SUCCEEDED", "FAILED", "CANCELLED"]:
            break
        
        time.sleep(.1)  # Check every .1 seconds

    if state != "SUCCEEDED":
        failure_reason = status["QueryExecution"]["Status"].get("StateChangeReason", "Unknown Error")
        raise Exception(f"Athena query failed with state: {state}, Reason: {failure_reason}")


    # Get Query Results
    results = athena_client.get_query_results(QueryExecutionId=query_execution_id)

    columns = [col["Label"] for col in results["ResultSet"]["ResultSetMetadata"]["ColumnInfo"]]

    # Extract Rows
    rows = []
    for row in results["ResultSet"]["Rows"][1:]:  # Skip header row
        extracted_row = [col.get("VarCharValue", None) for col in row["Data"]]  # Extract actual values
        rows.append(extracted_row)

    # Convert to Pandas DataFrame
    df = pd.DataFrame(rows, columns=columns)
    df = df.fillna("NA")

    return df

In [65]:
def weighted_avg(df, col, gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, inte = None):


    gb2 = gb1 + gb2
    gb3 = gb2 + gb3
    gb4 = gb3 + gb4

    average_gb1 = df[col].iloc[:gb1].mean()
    weighted_gb1 = average_gb1 * weight1

    average_gb2 = df[col].iloc[gb1:gb2].mean()
    weighted_gb2 = average_gb2 * weight2


    average_gb3 = df[col].iloc[gb2:gb3].mean()
    weighted_gb3 = average_gb3 * weight3

    average_gb4 = df[col].iloc[gb3:gb4].mean()
    weighted_gb4 = average_gb4 * weight4


    weighted_avg = round(((weighted_gb1 + weighted_gb2 + weighted_gb3 + weighted_gb4) / sum([weight1, weight2, weight3, weight4])), 3)

    
    if inte == 1:
        weighted_avg = int(weighted_avg)

    return weighted_avg

## This function that return the teams first game, and the oldest game that can be used for Barry

In [43]:
def oldest_usable_game(team, games_back):
    query = f"""
        WITH ordered_games AS (
            SELECT date,
                ROW_NUMBER() OVER (PARTITION BY team ORDER BY date ASC) AS row_num
            FROM "{DATABASE}"."{TABLE}"
            WHERE team = '{team}' 
        )
        SELECT date
        FROM ordered_games
    """
    df = query_athena_df(query)
    df.columns = df.columns.str.replace('_', ' ').str.title()  # Format column names
    df['Date'] = pd.to_datetime(df['Date'])  # Ensure 'Date' is in datetime format
    df = df.sort_values(by='Date', ascending=True)  # Sort from oldest to newest

    teams_first_game = str(df['Date'].iloc[(0)])[:10]
    oldest_game_for_modeling = str(df['Date'].iloc[(games_back)])[:10]
    
    messege = f'Barry reserves the first 34 games of a teams history to use for modeling. The first superbowl era game {team} played was on {teams_first_game}, chose a date equal to or more recent than {oldest_game_for_modeling}.'
    # the model used depends on the date due to missing data. 


    return team, teams_first_game, oldest_game_for_modeling, messege

In [44]:
team, teams_first_game, oldest_game_for_modeling, messege = oldest_usable_game('HOU', 34)
messege

'Barry reserves the first 34 games of a teams history to use for modeling. The first superbowl era game HOU played was on 2002-09-08, chose a date equal to or more recent than 2004-09-26.'

## Train a new model that drops downs_3D_att	downs_3D_con	downs_4D_att	downs_4D_con	game_time_off becauser this data is missing in games begining 1991. So if the date for a team is before 1994 the code will defualt to this second smaller model. 

##### I am using the training set created in the 'barry_02212025.ipynb' script. i just removes the time of possesion and cluth columns 4 total. 2 off 2 def. 

In [48]:
model_2 = pd.read_csv('aggregated_stats_modeling_removed_featues.csv')

In [79]:
model_2 

,points,passing_yds,passing_int,passing_times_sacked,rushing_yds,fmb,punts_yds,penalty_yds,def_passing_yds,def_passing_int,...,def_rushing_yds,def_fmb,pass_play_percentage,def_pass_play_percentage,drives,def_drives,tds_per_yard,def_tds_per_yard,fg_percentage,home_game
0,9,130.390,1.072,2.770,120.300,1.131,268.253,57.167,193.517,1.098,...,89.181,0.847,47.695,55.853,9.554,8.364,48.955,55.486,52.669,1
1,26,130.668,0.903,2.696,121.437,0.849,256.991,52.673,180.224,0.619,...,126.402,0.609,48.000,47.742,9.430,9.431,48.492,65.779,61.184,1
2,10,131.860,0.952,2.686,118.405,0.898,266.543,52.295,220.878,1.350,...,113.832,0.760,48.579,54.146,9.607,8.622,48.103,70.696,59.025,1
3,14,132.273,1.169,1.638,131.074,0.610,226.376,41.846,199.622,1.285,...,146.990,0.629,47.094,48.406,8.470,9.106,40.777,70.031,69.603,1
4,24,132.276,1.027,2.721,115.766,0.974,267.350,52.826,186.404,0.605,...,126.514,0.707,49.283,48.904,9.555,9.446,46.354,62.414,56.693,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10817,31,324.114,1.080,1.648,109.169,0.272,198.134,53.776,218.148,1.037,...,125.005,0.682,62.070,51.483,9.252,10.207,77.213,58.294,73.334,0
10818,16,324.790,1.032,1.503,111.804,0.352,198.991,55.949,267.263,0.932,...,107.553,0.588,61.738,57.522,9.194,9.504,78.564,76.311,74.040,1
10819,14,325.112,1.092,1.633,111.139,0.270,165.150,59.412,228.781,0.709,...,132.238,0.578,63.030,51.463,8.857,9.934,81.823,66.097,73.597,1
10820,39,325.238,0.809,1.209,104.625,0.467,199.777,65.375,211.953,0.991,...,111.499,0.594,60.627,55.773,9.909,8.479,89.139,51.250,60.739,0


In [83]:
## outliers
def handle_outliers(df, feature):

    # calculate the mean and standard deviation of the feature
    mean = df[feature].mean()
    std = df[feature].std()

    # define the threshold for outliers (3 standard deviations)
    threshold = 3 * std

    # save the indices of outliers
    outlier_indices = df[(df[feature] < mean - threshold) | (df[feature] > mean + threshold)].index
   
   # replace outliers with NaN values
    #df.loc[outlier_indices, feature] = np.nan
    #print(f"Number of rows dropped for feature '{feature}': {len(outlier_indices)}")

    # you can also remove outliers from the DataFrame completely
    df = df.drop(outlier_indices)

    return df

In [92]:
def preprocessing(df, target):

    # outliers
    for feature in df.select_dtypes(include=[np.number]).columns:
        df = handle_outliers(df, feature)
    print(target)
    # partitioning
    X = df.drop(columns=[target])
    y = df[target]
    X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)

    # scaling
    standard_scaler = StandardScaler()
    X_train = pd.DataFrame(standard_scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_val   = pd.DataFrame(standard_scaler.transform(X_val), columns=X_val.columns, index=X_val.index)
    X_test  = pd.DataFrame(standard_scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    scaler_filename = 'chalk_22_scaler_2.pkl'
    joblib.dump(standard_scaler, scaler_filename)


    # feature importance
    #model = RandomForestRegressor(random_state=42)
    #clf = model.fit(X_train, y_train)
    #feature_importance(clf, X_train)

    # feature independece
    numerical_cols = X_test.select_dtypes(include=[float, int])
    results_list = []
    # iterate over all combinations of numerical columns
    for i, col1 in enumerate(numerical_cols.columns):
        for col2 in numerical_cols.columns[i+1:]:
            x = numerical_cols[col1]
            y = numerical_cols[col2]
            # calculate Pearson's correlation coefficient and p-value
            corr_coefficient, p_value = pearsonr(x, y)
            # append the results to the list
            results_list.append({'Variable1': col1, 'Variable2': col2, 'Correlation Coefficient': corr_coefficient, 'P-Value': p_value})
 
    # convert the list to a DataFrame
    results_df = pd.DataFrame(results_list)
    results_df = results_df.sort_values(by='Correlation Coefficient', ascending=False)

    return X_train, X_test, y_train, y_test, X_val, y_val, results_df

In [93]:
df = model_2
target = 'points'
X_train, X_test, y_train, y_test, X_val, y_val, results_df = preprocessing(df, target)

points


In [94]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import joblib

# Fit the model
model = LinearRegression()
model.fit(X_train, y_train)
joblib.dump(model, 'lr_model_2.joblib')

['lr_model_2.joblib']

In [114]:
# Load the scaler
scaler = joblib.load("chalk_22_scaler_2.pkl")

# Check if it's a StandardScaler object
print(type(scaler))  

# Try accessing feature-related attributes
if hasattr(scaler, "n_features_in_"):
    print(f"Scaler expects {scaler.n_features_in_} features.")  # Only works for newer versions of scikit-learn
else:
    print("Scaler does not have `n_features_in_`, checking alternative methods.")

# Check expected shape
print(f"Expected number of features: {scaler.mean_.shape[0]}")  # Mean vector length = number of features


<class 'sklearn.preprocessing._data.StandardScaler'>
Scaler expects 20 features.
Expected number of features: 20


# Use the model
## Create dataframe one row per team

In [195]:
def collect_df_for_each_matchup(team, opponent, date1, date2, model):
    
    # create data frames with the 35 most recent games =< the proivided date for both team offense and opponent defense
    query_offense = f'''
        SELECT date, team, opponent, points, home_game,
            passing_yds, passing_tds, passing_int, 
            passing_times_sacked, rushing_yds, 
            rush_tds, fmb, "3d_att", "3d_conversions", "4d_att", "4d_conversions", time_of_possession, 
            fga, punts_yds, punts_total, "2pm", "2pa",
            penalty_yds, fgm, passing_att, rushing_att
        FROM "{DATABASE}"."{TABLE}"
        WHERE team = '{team}'
            AND TRY_CAST(DATE_PARSE(date, '%m/%d/%Y') AS DATE) <= DATE '{date1}'
        ORDER BY TRY_CAST(DATE_PARSE(date, '%m/%d/%Y') AS DATE) DESC
        LIMIT {games_back+1};
    '''
    off_df = query_athena_df(query_offense)

    query_defense = f'''
        SELECT 
            date, team, opponent, passing_yds AS def_passing_yds, 
            passing_tds AS def_passing_tds, passing_int AS def_passing_int, 
            passing_times_sacked AS def_passing_times_sacked, rushing_yds AS def_rushing_yds, 
            rush_tds AS def_rush_tds, fmb AS def_fmb, 
            "3d_att" AS def_3d_att, "3d_conversions" AS def_3d_conversions, 
            "4d_att" AS def_4d_att, "4d_conversions" AS def_4d_conversions, 
            time_of_possession AS def_time_of_possession, fga AS def_fga, 
            punts_total AS def_punts_total, "2pm" AS def_2pm, 
            "2pa" AS def_2pa, passing_att AS def_passing_att, rushing_att AS def_rushing_att
        FROM "{DATABASE}"."{TABLE}"
        WHERE opponent = '{opponent}'
            AND TRY_CAST(DATE_PARSE(date, '%m/%d/%Y') AS DATE) <= DATE '{date2}' 
        ORDER BY TRY_CAST(DATE_PARSE(date, '%m/%d/%Y') AS DATE) DESC
        LIMIT {games_back+1};
    '''
    def_df = query_athena_df(query_defense)


    # join the two dfs on the index column because teams may play on different days on the same week. 
    merged_df = off_df.merge(def_df, left_index=True, right_index=True, how='inner')
    merged_df = merged_df[merged_df['date_x'] != '1/2/2023']
    merged_df = merged_df[merged_df['date_y'] != '1/2/2023']

    # first take out any information from the row date in question
    home_game = int(merged_df['home_game'][0])

    # remove the top row because you dont want to train the model on data from the same week we are uses for points
    merged_df_2 = merged_df.drop(merged_df.index[0])
    merged_df_2 = merged_df_2.drop(columns=['date_x', 'team_x', 'opponent_x', 'date_y', 'team_y', 'opponent_y', 'home_game', 'points'])

    if model == 2:
        # drop these for the smaller model
        merged_df_2 = merged_df_2.drop(columns=['time_of_possession', 'def_time_of_possession', '2pm', '2pa', 'def_2pm', 'def_2pa',
            '3d_att', 'def_3d_att', '3d_conversions', 'def_3d_conversions', '4d_att', 'def_4d_att', '4d_conversions', 
            'def_4d_conversions'])

    # convert to integers. this is the part that despises N/A values 
    merged_df_2 = merged_df_2.astype(int)


    
    return(merged_df_2, home_game)

In [196]:
def create_features(df, model):  

    merged_df_2 = df
    ##### pass_play_percentage
    merged_df_2['pass_play_percentage'] = 100*(merged_df_2['passing_att'] / (merged_df_2['passing_att'] + merged_df_2['rushing_att']))
    merged_df_2['def_pass_play_percentage'] = 100*(merged_df_2['def_passing_att'] / (merged_df_2['def_passing_att'] + merged_df_2['def_rushing_att']))

    merged_df_2 = merged_df_2.drop(columns=['passing_att', 'rushing_att', 'def_passing_att', 'def_rushing_att'])

    ##### drives
    # Offensive drives
    merged_df_2['drives'] = merged_df_2['passing_tds'] + merged_df_2['rush_tds'] + merged_df_2['fga'] + merged_df_2['punts_total']

    # Defensive drives
    merged_df_2['def_drives'] = merged_df_2['def_passing_tds'] + merged_df_2['def_rush_tds'] + merged_df_2['def_fga'] + merged_df_2['def_punts_total']

    # drop
    merged_df_2 = merged_df_2.drop(columns=['punts_total', 'def_punts_total', 'def_fga'])

    ##### tds per 10000 yards
    # Offensive touchdowns per yard
    merged_df_2['tds_per_yard'] = 10000 * ((merged_df_2['passing_tds'] + merged_df_2['rush_tds']) / \
                                (merged_df_2['passing_yds'] + merged_df_2['rushing_yds']))

    # Defensive touchdowns per yard
    merged_df_2['def_tds_per_yard'] = 10000 * ((merged_df_2['def_passing_tds'] + merged_df_2['def_rush_tds']) / \
                                    (merged_df_2['def_passing_yds'] + merged_df_2['def_rushing_yds']))

    merged_df_2 = merged_df_2.drop(columns=['passing_tds', 'rush_tds', 'def_passing_tds', 'def_rush_tds'])


    ##### fg_percentage
    merged_df_2['fg_percentage'] = 100*(np.where(merged_df_2['fga'] == 0, 0, merged_df_2['fgm'] / merged_df_2['fga']))

    # Now drop 'fgm' and 'fga'
    merged_df_2 = merged_df_2.drop(columns=['fgm', 'fga'])

    merged_df_2 = merged_df_2.astype(int)


    if model == 1:
        ##### cluth metric
        # Offensive clutch conversion percentage
        merged_df_2['clutch_conversion_percentage'] = 100*((merged_df_2['3d_conversions'] + merged_df_2['4d_conversions'] + merged_df_2['2pm']) / \
                                                    (merged_df_2['3d_att'] + merged_df_2['4d_att'] + merged_df_2['2pa']))

        # Defensive clutch conversion percentage
        merged_df_2['def_clutch_conversion_percentage'] = 100*(1 - ((merged_df_2['def_3d_conversions'] + merged_df_2['def_4d_conversions'] + merged_df_2['def_2pm']) / \
                                                        (merged_df_2['def_3d_att'] + merged_df_2['def_4d_att'] + merged_df_2['def_2pa'])))

        # Drop the original columns
        merged_df_2 = merged_df_2.drop(columns=[
            '3d_att', '4d_att', '2pa', '3d_conversions', '4d_conversions', '2pm',
            'def_3d_att', 'def_4d_att', 'def_2pa', 'def_3d_conversions', 'def_4d_conversions', 'def_2pm'
        ])

    return merged_df_2

In [197]:
def weighted(merged_df_2, model):    
    # Dictionary to store weighted averages
    weighted_averages = {}
    # List of columns to calculate weighted averages for
    if model == 2:
        columns = [
            'passing_yds', 'passing_int', 'passing_times_sacked', 'rushing_yds', 'fmb',
            'punts_yds', 'penalty_yds', 'def_passing_yds',
            'def_passing_int', 'def_passing_times_sacked',
            'def_rushing_yds', 'def_fmb', 'def_passing_times_sacked',
            'pass_play_percentage', 'def_pass_play_percentage', 'drives',
            'def_drives', 'tds_per_yard', 'def_tds_per_yard',
            'fg_percentage'
        ]

    else:
        columns = [
            'passing_yds', 'passing_int', 'passing_times_sacked', 'rushing_yds', 'fmb', 'time_of_possession',
            'punts_yds', 'penalty_yds', 'def_passing_yds',
            'def_passing_int', 'def_passing_times_sacked',
            'def_rushing_yds', 'def_fmb', 'def_time_of_possession', 'def_passing_times_sacked',
            'pass_play_percentage', 'def_pass_play_percentage', 'drives',
            'def_drives', 'tds_per_yard', 'def_tds_per_yard',
            'clutch_conversion_percentage', 'def_clutch_conversion_percentage',
            'fg_percentage'
            ]

    # Calculate weighted averages and store in the dictionary
    for col in columns:
        weighted_averages[col] = weighted_avg(merged_df_2, col, *weights)

    # Convert dictionary to a DataFrame (single-row)
    weighted_avg_df = pd.DataFrame([weighted_averages])

    return weighted_avg_df

In [ ]:
def get_predictions_2(team, opponent, date1, date2):  

    team = team
    opponent = opponent
    date1 = date1
    date2 = date2


    if datetime.strptime(date1, '%Y-%m-%d').year < 1994 or datetime.strptime(date2, '%Y-%m-%d').year < 1994:
        model_used = 2
    else:
        model_used = 1
    print(model_used)



    # create a 34 most recent game df fro each matchup
    merged_df_2, home_game = collect_df_for_each_matchup(team, opponent, date1, date2, model_used)

    # create features
    merged_df_2 = create_features(merged_df_2, model_used)

    # get the aggregated weighted averages of each feature
    weighted_avg_df = weighted(merged_df_2, model_used)
    weighted_avg_df.columns

    # add home_game the only varible that doesnt need to be weighted
    weighted_avg_df['home_game'] = home_game


    if model_used == 2 :
        # scale the featurs using the scaler form the model
        scaler = joblib.load("chalk_22_scaler_2.pkl")
        scaled_inputs = scaler.transform(weighted_avg_df)
        # model_2_training = pd.DataFrame(scaled_inputs, columns= model_2_training.columns)
        model_lr = joblib.load("lr_model_2.joblib")

    else:
        # scale the featurs using the scaler form the model
        scaler = joblib.load("chalk_22_scaler.pkl")
        scaled_inputs = scaler.transform(weighted_avg_df)
        # model_2_training = pd.DataFrame(scaled_inputs, columns= model_2_training.columns)
        model_lr = joblib.load("lr_model.joblib")


    # Make predictions for each row
    predictions = model_lr.predict(scaled_inputs)

    return predictions, model_used

In [259]:
def model_output(team, opponent, points_team1, points_team2):    
    sd = 8
    limit = 100
    n = 0
    team1_wins = 0
    team2_wins = 0

    # Create an empty DataFrame to store results
    df = pd.DataFrame(columns=[team, opponent])

    while n < limit:
        # Add some variance to the scores
        team1_score =  round(np.random.normal(loc=points_team1.item(), scale=sd), 1)
        team2_score =  round(np.random.normal(loc=points_team2.item(), scale=sd), 1)

        # Append the scores to the DataFrame
        df.loc[len(df)] = [team1_score, team2_score]

        if team1_score > team2_score:
            team1_wins += 1
        else:
            team2_wins += 1

        n += 1

    team1_win_pct = team1_wins / limit

    return team1_win_pct, df

In [276]:
def week_to_date(team, season, week):
    original_week = week
    direction = -1  # Start by decrementing
    
    while 1 <= week <= 18:  # Ensure week stays within valid range
        query_date_from_week = f'''
            SELECT date
            FROM "{DATABASE}"."{TABLE}"
            WHERE team = '{team}' AND season = {season} AND week = {week}
        '''
        date_from_week = query_athena_df(query_date_from_week)
        
        if not date_from_week.empty:  # If the query returns a result, return it

            date_from_week["date"] = pd.to_datetime(date_from_week["date"]).dt.strftime("%Y-%m-%d")

            return date_from_week["date"].iloc[0]  # Returns '2002-10-06'

        
        week += direction  # Move in the current direction
        
        if week == 0:  # If we hit week 0, switch direction to increment
            week = original_week + 1
            direction = 1  # Start incrementing instead
        
    return None  # Return None if no valid date is found within week 1-18



## Inputs

In [280]:
# Save row 1 column values into individual variables before dropping
team = 'KAN'
opponent = 'HOU'
season1 = 2002
week1 = 5
season2 = 2002
week2 = 9
date1 = week_to_date(team, season1, week1)
date2 = week_to_date(opponent, season2, week2)

#date1 = '2010-10-08'
#date2 = '2006-10-08'

In [281]:
date1

'2002-10-06'

In [282]:
date2

'2002-11-03'

In [283]:
team, teams_first_game, oldest_game_for_modeling, messege = oldest_usable_game(team, 34)
opponent, teams_first_game_2, oldest_game_for_modeling_2, messege_2 = oldest_usable_game(opponent, 34)

# Convert input dates and oldest usable game dates to datetime objects
date1_dt = datetime.strptime(date1, '%Y-%m-%d')
date2_dt = datetime.strptime(date2, '%Y-%m-%d')
oldest_game_for_modeling_dt = datetime.strptime(oldest_game_for_modeling, '%Y-%m-%d')
oldest_game_for_modeling_2_dt = datetime.strptime(oldest_game_for_modeling_2, '%Y-%m-%d')


# Check if either date is before the oldest usable game for that team
date1_too_early = date1_dt < oldest_game_for_modeling_dt
date2_too_early = date2_dt < oldest_game_for_modeling_2_dt

if date1_too_early:
    print(messege)
if date2_too_early:
    print(messege_2)

# Only run predictions if BOTH dates are valid
if not date1_too_early and not date2_too_early:
    points_team1, model_used1 = get_predictions_2(team, opponent, date1, date2)
    points_team2, model_used2 = get_predictions_2(opponent, team, date2, date1)
    team1_win_pct, df = model_output(team, opponent, points_team1, points_team2)

Barry reserves the first 34 games of a teams history to use for modeling. The first superbowl era game HOU played was on 2002-09-08, chose a date equal to or more recent than 2004-09-26.


In [265]:
df

,KAN,NWE
0,26.2,29.8
1,14.7,21.0
2,3.9,14.1
3,24.8,26.9
4,17.7,31.4
...,...,...
95,25.7,30.1
96,11.9,33.2
97,11.5,38.5
98,12.4,20.1


## create a table for dynamono db that has rows date, week, season, team, opponent, then all the required rows for modeling already averaged out for games back